# Caso Práctico - Visualización de Datos
---
Visualización de Datos del Máster Universitario en Ingeniería y Ciencia de Datos (2023/2024)

Alumno: **Javier Orive Soto**


# 1. Importación de dependencias

In [1]:
import pandas as pd

import numpy as np
import altair as alt

# 2. Importación de datos

## 2.1 Datos económicos

In [2]:
# Cargar el archivo Excel
file_path = 'Datos/Economico/PrecioMedioHorarioFinal_2023_d.xlsx'
df = pd.read_excel(file_path)

# Seleccionar solo las columnas de interés
df = df[['name', 'value', 'datetime']]

In [3]:
# Convertir la columna datetime a tipo datetime
df['datetime'] = pd.to_datetime(df['datetime'], utc = True)

# Convertir la columna value a tipo float
df['value'] = df['value'].astype(float)

# Convertir la columna name a tipo categoría
df['name'] = df['name'].astype('category')


In [4]:
df.head()

,name,value,datetime
0,Precio medio horario final suma de componentes,981.00,2022-12-31 23:00:00+00:00
1,Precio medio horario final suma de componentes,3389.78,2023-01-01 23:00:00+00:00
2,Precio medio horario final suma de componentes,3726.12,2023-01-02 23:00:00+00:00
3,Precio medio horario final suma de componentes,3279.46,2023-01-03 23:00:00+00:00
4,Precio medio horario final suma de componentes,3095.54,2023-01-04 23:00:00+00:00


In [5]:
df.info

<bound method DataFrame.info of                                                    name    value  \
0        Precio medio horario final suma de componentes   981.00   
1        Precio medio horario final suma de componentes  3389.78   
2        Precio medio horario final suma de componentes  3726.12   
3        Precio medio horario final suma de componentes  3279.46   
4        Precio medio horario final suma de componentes  3095.54   
...                                                 ...      ...   
6200  Precio medio horario componente servicio de in...     0.00   
6201  Precio medio horario componente servicio de in...     0.00   
6202  Precio medio horario componente servicio de in...     0.00   
6203  Precio medio horario componente servicio de in...     0.00   
6204  Precio medio horario componente servicio de in...     0.00   

                      datetime  
0    2022-12-31 23:00:00+00:00  
1    2023-01-01 23:00:00+00:00  
2    2023-01-02 23:00:00+00:00  
3    2023-01-03 23:

In [6]:
df = df.pivot(index='datetime', columns='name', values='value').reset_index()

In [7]:
df

name,datetime,Precio medio horario componente banda secundaria,Precio medio horario componente control factor potencia,Precio medio horario componente desvíos medidos,Precio medio horario componente fallo nominación UPG,Precio medio horario componente incumplimiento energía de balance,Precio medio horario componente mecanismo de ajuste RD-L 10/2022,Precio medio horario componente mercado diario,Precio medio horario componente mercado intradiario,Precio medio horario componente pago de capacidad,Precio medio horario componente reserva de potencia adicional a subir,Precio medio horario componente restricciones PBF,Precio medio horario componente restricciones intradiario,Precio medio horario componente restricciones tiempo real,Precio medio horario componente saldo P.O.14.6,Precio medio horario componente saldo de desvíos,Precio medio horario componente servicio de interrumpibilidad,Precio medio horario final suma de componentes
0,2022-12-31 23:00:00+00:00,4.986,-0.08,1.115,0.0,-0.070,47.08,6.697,-0.066,0.000,0.0,25.244,0.0,2.077,0.110,-1.100,0.0,981.00
1,2023-01-01 23:00:00+00:00,4.795,-0.08,0.393,0.0,-0.083,137.69,112.550,0.767,0.433,0.0,7.600,0.0,8.464,0.063,0.603,0.0,3389.78
2,2023-01-02 23:00:00+00:00,1.911,-0.08,0.183,0.0,-0.139,160.66,138.787,-0.343,0.429,0.0,2.318,0.0,5.273,0.061,0.162,0.0,3726.12
3,2023-01-03 23:00:00+00:00,2.787,-0.08,0.297,0.0,-0.142,235.00,115.257,-0.179,0.429,0.0,3.572,0.0,5.004,0.085,-0.177,0.0,3279.46
4,2023-01-04 23:00:00+00:00,2.392,-0.08,0.355,0.0,-0.108,220.40,109.185,-0.367,0.429,0.0,3.515,0.0,4.527,0.011,-0.062,0.0,3095.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-12-26 23:00:00+00:00,1.427,-0.09,0.638,0.0,-0.518,0.00,79.545,0.073,0.435,0.0,2.199,0.0,4.385,0.013,-0.498,0.0,2102.64
361,2023-12-27 23:00:00+00:00,1.503,-0.09,0.358,0.0,-0.546,0.00,98.193,0.161,0.435,0.0,1.949,0.0,3.242,0.056,0.211,0.0,2531.30
362,2023-12-28 23:00:00+00:00,1.005,-0.09,0.315,0.0,-0.359,0.00,103.500,-0.067,0.435,0.0,2.144,0.0,2.820,0.043,0.122,0.0,2636.84
363,2023-12-29 23:00:00+00:00,2.215,-0.09,0.463,0.0,-0.128,0.00,68.405,-0.143,0.000,0.0,5.833,0.0,0.425,0.001,-0.222,0.0,1842.23


In [8]:
alt.Chart(df).mark_line().encode(
  x='datetime:T',
  y='Precio medio horario componente banda secundaria:Q'
)

alt.Chart(...)

In [9]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [10]:
alt.Chart(df).mark_line().encode(
  x='datetime:T',
  y='value:Q',
  color='name:N'
)

alt.Chart(...)

In [11]:
alt.Chart(df).mark_bar().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N'
)

alt.Chart(...)

: 

In [12]:
selection = alt.selection_point(fields=['name'], bind='legend')

AttributeError: module 'altair' has no attribute 'selection_point'

In [13]:
# Plotting
chart.mark_line().encode(
    x='datetime:T',
    y='value:Q',
    tooltip=['datetime:T', 'value:Q']
).properties(
    title='Hourly Average Price Over Time',
    width=800,
    height=400
).interactive()





NameError: name 'chart' is not defined

In [14]:
chart.show

NameError: name 'chart' is not defined

In [71]:
# Crear el gráfico apilado
chart.mark_area().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N',
    tooltip=['datetime:T', 'name:N', 'value:Q']
).properties(
    title='Evolución del Precio Horario de la Electricidad en 2023 por Componentes',
    width=800,
    height=400
).interactive()


c:\Users\javie\anaconda3\envs\VD\lib\site-packages\altair\vegalite\v4\api.py:101: UserWarning: data of type <class 'pandas.core.groupby.generic.DataFrameGroupBy'> not recognized
  warnings.warn("data of type {} not recognized".format(type(data)))


SchemaValidationError: Invalid specification

        altair.vegalite.v4.api.Chart->0, validating 'anyOf'

        <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001FF6B4D0880> is not valid under any of the given schemas
        

alt.Chart(...)

In [72]:
# Crear el gráfico apilado
alt.Chart(df).mark_area().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N',
    tooltip=['datetime:T', 'name:N', 'value:Q']
).properties(
    title='Evolución del Precio Horario de la Electricidad en 2023 por Componentes',
    width=800,
    height=400
).interactive()

c:\Users\javie\anaconda3\envs\VD\lib\site-packages\altair\vegalite\v4\api.py:101: UserWarning: data of type <class 'pandas.core.groupby.generic.DataFrameGroupBy'> not recognized
  warnings.warn("data of type {} not recognized".format(type(data)))


SchemaValidationError: Invalid specification

        altair.vegalite.v4.api.Chart->0, validating 'anyOf'

        <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001FF6B4D0880> is not valid under any of the given schemas
        

alt.Chart(...)